In [1]:
import tensorflow as tf
tf.__version__

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


'2.5.0-rc1'

In [4]:
import pandas as pd
pd.read_csv('./data/Train/Bangkok_imputed.csv', index_col=0)

,PM2.5,Temp(C),WindDir,Wind Speed(km/h)
2016-03-03 08:00:00,62.9,26.4,65.0,15.0
2016-03-03 09:00:00,62.9,26.4,65.0,15.0
2016-03-03 10:00:00,55.5,31.4,75.0,13.0
2016-03-03 11:00:00,55.5,31.4,75.0,13.0
2016-03-03 12:00:00,47.9,31.4,75.0,13.0
...,...,...,...,...
2019-03-17 19:00:00,42.2,31.3,70.0,17.0
2019-03-17 20:00:00,41.2,31.3,70.0,17.0
2019-03-17 21:00:00,37.7,31.3,70.0,17.0
2019-03-17 22:00:00,39.0,30.2,85.0,19.0


In [20]:
import pandas as pd

provinces = ['Bangkok','Chanthaburi','Chiang Mai','Kanchanaburi','Khon Kaen','Songkhla']

df = pd.read_csv('./data/Train/Bangkok_imputed.csv')
print(df.shape)
print(df.isnull().sum())
df.head()

(8878, 4)
Temp          0
WindDir       0
Wind Speed    0
PM2.5         0
dtype: int64


,Temp,WindDir,Wind Speed,PM2.5
2016-03-03 07:00:00,26.4,65,15,62.900000
2016-03-03 10:00:00,31.4,75,13,52.966667
2016-03-03 13:00:00,34.1,70,12,35.266667
2016-03-03 16:00:00,34.6,60,12,33.050000
2016-03-03 19:00:00,29.1,70,11,42.066667


In [21]:
from sklearn.preprocessing import MinMaxScaler

pm = df['PM2.5']

scaler = MinMaxScaler().fit(pm)

train_scaled = scaler.transform(pm)

In [22]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# define generator
n_input = 2921
n_features = 1
generator = TimeseriesGenerator(train_scaled, train_scaled, length=n_input, batch_size=100)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

# define model
model = Sequential([
    LSTM(100, activation='relu', input_shape=(n_input, n_features)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(generator,epochs=5, batch_size=100)

Test

In [31]:
df.head(4)

,Temp,WindDir,Wind Speed,PM2.5
2016-03-03 07:00:00,26.4,65,15,62.900000
2016-03-03 10:00:00,31.4,75,13,52.966667
2016-03-03 13:00:00,34.1,70,12,35.266667
2016-03-03 16:00:00,34.6,60,12,33.050000


In [25]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [28]:

# define generator
n_input = 2921
n_features = 1
generator = TimeseriesGenerator(x, y, length=2, batch_size=1)

In [33]:
X, Y = generator[1]

In [34]:
X

array([[[31.4, 75. , 13. ],
        [34.1, 70. , 12. ]]])

In [35]:
Y

array([33.05])